![CIn UFPE](https://media.licdn.com/dms/image/C4E1BAQGE9NakMenMTA/company-background_10000/0/1625683462457/cinufpe_cover?e=2147483647&v=beta&t=TQcLMZkvhvLF3e-QtAjjzrNGfYIMPJoYikrrhZfJTVA)
# IN1165 - Tomada de Decisão com Machine Learning
## Roberto de Medeiros <rmff2@cin.ufpe.br>
## Referência:
#### [1] Kochenderfer, Mykel (2015). Decision Making Under Uncertainty. MIT Press. Cambridge, Massachusetts.  Disponível em http://web.stanford.edu/group/sisl/public/dmu.pdf. Acesso em 19 de abr. de 2024.
#### [2] SISL. SISL GitHub. Disponível em https://github.com/sisl/CollisionAvoidancePOMDPs.jl . Acesso em 19 de abr. de 2024.

In [32]:
import random

#### Vetores (variáveis)

In [38]:
h = [] #float, altitude do intruso versus o avião
h0 = [] #float, taxa vertical do avião
h1 = [] #float, taxa vertical do intruso
tau = [] #float, tempo de potencial colisão
s_adv = [] #str, advertência
s_res = [] #bool, resposta do piloto
#atributos temporais
ht_0 = [] #float, h¨0 ft/s2
ht_1 = [] #float, h¨1 ft/s2
a = '' #nova advertência
#atributos extras
d = [] #float, distância para colisão
v0 = 25.0 #float, velocidade do avião ft/s, será mantida cte de 1500ft/min = 25ft/s
v1 = 25.0 #float, velocidade do intruso ft/s, será mantida cte de 1500ft/min = 25ft/s
rwd = [] #float, recompensa

#### Funções

In [34]:
def advisiory(s_res,s_adv,a):
    return 'COC',0.0#precisa ser feito com diferentes situações, inicial é livre de colisões e 0.0

def rnd_adversary(h,v):
    rnd = random.randint(0,1)
    a = ''
    if(rnd == 0):
        rnd = -1 #se 0 é negativo (-1), 1 senão
        a = 'DENCEND' #descer (25ft/s), sem aceleração
        return 1.0*(h-v),a
    else:
        a = 'CLIMB' #subir (25ft/s), sem aceleração
        return 1.0*(h+v),a

def rnd_move():
    rnd = random.randint(0,1)
    if(rnd == 0):
        return 'DENCEND'
    else:
        return 'CLIMB'

def advisory_make(h,h1_moving):#h1_moving é o movimento str do adversário
    #if h == 0.0:
    if h1_moving=='CLIMB':
        return 'DESCEND'
    elif h1_moving=='DESCEND':
        return 'CLIMB'
    elif h1_moving=='NORMAL':
        return rnd_move()
    return 'COC'
    #else:
        #return 'COC'

#### Inicializando

In [39]:
h0.append(35000.0) #altitude inicial segura é 30.000 a 40.000 ft
h1.append(35000.0) #intruso em mesma altitude inicial (atribuição pessoal)
h.append(0.0)
#d.append(60695-5.381) #606955,381 ft é a distância mínima média dos radares atuais
d.append(500) #606955,381 ft é a distância mínima média dos radares atuais
#v0 = v1 = cte 25ft/s, velocidade usada também em subida ou descida, desconsiderou aceleração, a princípio
tau.append((d[0])/50.0) #distância / soma da velocidade de ambos aviões
s_adv.append('COC')
rwd.append(0.000000001) #COC = 1*(10^-9)
s_res.append(True) #(?) piloto em resposta, a princípio
#a terá inicial aleatório
rnd = random.randint(0,1)
if(rnd == 0):
    rnd = -1 #se 0 é negativo (-1), 1 senão
    a = 'DENCEND' #descer (-25ft/s), sem aceleração
else:
    a = 'CLIMB' #subir (25ft/s), sem aceleração

print('h0 ')
print(h0)
print('h1 ')
print(h1)
print('h ')
print(h)
print('d ')
print(d)
print('tau ')
print(tau)
print('s_adv ')
print(s_adv)
print('rwd ')
print(rwd)
print('s_res ')
print(s_res)

h0 
[35000.0]
h1 
[35000.0]
h 
[0.0]
d 
[500]
tau 
[10.0]
s_adv 
['COC']
rwd 
[1e-09]
s_res 
[True]


#### Atualizando valores

In [40]:
repeat = True
h1_count = 5
h1_move = -25.0
h1_moving = 'NORMAL'
collision = v0+v1#irão colidir se estiverem a v0+v1 ft, porque o próximo movimento é bate ou LOLO
it_count = 0

while(repeat):
    if h1_count==5:#rand a cada 5s | pode ser a cada iteração
        h1_count=0
        h1_move, h1_moving = rnd_adversary(0.0,25.0)
    else:
        h1_moving = 'NORMAL'
    h1.append(h1_move)
    h1_count+=1
    if(a=='DESCEND'):
        h0.append(1.0*(h0[len(h0)-1])-25.0)
    elif(a=='CLIMB'):
        h0.append(1.0*(h0[len(h0)-1])+25.0)
    else:
        h0.append(h0[len(h0)-1])
    print(f'h0 {h0[len(h0)-1]}')
    d.append(d[len(d)-1]-v0-v1)
    print(f'd {d[len(d)-1]}')
    s_adv.append(a)
    s_res.append(True)
    h = abs(h0[len(h0)-1]-h1[len(h1)-1])#diferença entre ambas alturas
    #update no 'a' decisão da torre de controle
    a = advisory_make(h,h1_moving)
    #if it_count%100 == 0:
    print('adv: '+a)
    print('h1: '+h1_moving)
    it_count += 1
    #parada em 1 iteração
    if d[len(d)-1]<collision:
        if s_adv[len(s_adv)-1] == 'COC' and a == 'COC':
            print('SURVIVED!')
        else:
            print('LOLO')
        print('END!')
        repeat = False

h0 35025.0
d 450.0
adv: COC
h1: DENCEND
h0 35025.0
d 400.0
adv: CLIMB
h1: NORMAL
h0 35050.0
d 350.0
adv: CLIMB
h1: NORMAL
h0 35075.0
d 300.0
adv: DENCEND
h1: NORMAL
h0 35075.0
d 250.0
adv: DENCEND
h1: NORMAL
h0 35075.0
d 200.0
adv: COC
h1: DENCEND
h0 35075.0
d 150.0
adv: CLIMB
h1: NORMAL
h0 35100.0
d 100.0
adv: CLIMB
h1: NORMAL
h0 35125.0
d 50.0
adv: DENCEND
h1: NORMAL
h0 35125.0
d 0.0
adv: CLIMB
h1: NORMAL
LOLO
END!
